# Lecture 10: MCUNet & TinyML

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaurav-redhat/transformer_problems/blob/efficientml-course/efficientml_course/10_mcunet_tinyml/demo.ipynb)

Running ML on microcontrollers with 256KB RAM!


In [ ]:
!pip install torch -q
import torch
import torch.nn as nn

# MCU Memory Constraints
MCU_CONSTRAINTS = {
    'STM32F746': {'sram': 320, 'flash': 1024},  # KB
    'Arduino Nano 33': {'sram': 256, 'flash': 1024},
    'ESP32': {'sram': 520, 'flash': 4096},
}

def calculate_peak_memory(model, input_size, dtype_bytes=1):
    """Estimate peak memory during inference (activations)"""
    # Simplified: max activation size across layers
    x = torch.randn(1, *input_size)
    max_activation = 0
    
    def hook(module, input, output):
        nonlocal max_activation
        if isinstance(output, torch.Tensor):
            max_activation = max(max_activation, output.numel())
    
    hooks = []
    for module in model.modules():
        hooks.append(module.register_forward_hook(hook))
    
    with torch.no_grad():
        model(x)
    
    for h in hooks:
        h.remove()
    
    return max_activation * dtype_bytes / 1024  # KB

# Tiny model for MCU
class TinyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 8, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.classifier = nn.Linear(16, 10)
    
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x.view(x.size(0), -1))

model = TinyNet()
param_kb = sum(p.numel() for p in model.parameters()) / 1024
activation_kb = calculate_peak_memory(model, (3, 48, 48))

print("TinyNet for MCU:")
print(f"  Parameters: {param_kb:.1f} KB")
print(f"  Peak activation: {activation_kb:.1f} KB")
print(f"  Total: {param_kb + activation_kb:.1f} KB")
print(f"\n  Fits on STM32F746 (320KB SRAM)? {'Yes!' if param_kb + activation_kb < 320 else 'No'}")
print("\n🎯 TinyML requires extreme memory optimization!")
